In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd 
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import copy

from PIL import Image
import imageio

import statsmodels.api as sm
import statsmodels.formula.api as smf

## Load Data

In [ ]:
train_data = pd.read_csv('./data/train_discard_6_train.csv')
val_data = pd.read_csv('./data/train_discard_6_val.csv')
test_data = pd.read_csv('./train_discard_6_test.csv')

In [ ]:
print('Training data size: ' , len(train_data))
print('Validation data size: ' , len(val_data))
print('Testing data size: ' , len(test_data))

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
train_x = []
train_y = train_data['landmark_id']

for i in range(len(train_data)):
    image_id = train_data['id'][i]
    image_path = './data/train/' + image_id[0] + '/' + image_id[1] + '/' + image_id[2] +'/' + image_id + '.jpg'
    image = Image.open(image_path)
    
    train_x.append(preprocess(image))

In [ ]:
val_x = []
val_y = val_data['landmark_id']

for i in range(len(val_data)):
    image_id = val_data['id'][i]
    image_path = './data/train/' + image_id[0] + '/' + image_id[1] + '/' + image_id[2] +'/' + image_id + '.jpg'
    image = Image.open(image_path)
    
    val_x.append(preprocess(image))

In [ ]:
test_x = []
test_y = test_data['landmark_id']

for i in range(len(test_data)):
    image_id = test_data['id'][i]
    image_path = './data/train/' + image_id[0] + '/' + image_id[1] + '/' + image_id[2] +'/' + image_id + '.jpg'
    image = Image.open(image_path)

    test_x.append(preprocess(image))

In [ ]:
data = {
    'train': list(zip(train_x, train_y)),
    'val': list(zip(val_x, val_y)),
    'test': list(zip(test_x, test_y))
}

In [ ]:
dataloaders = { x: torch.utils.data.DataLoader(data[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val', 'test']}

In [ ]:
dataset_sizes = {
    'train': len(train_x),
    'val': len(val_x),
    'test': len(test_x)
}

## Model Training 

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            ctr = 0 
            for inputs, labels in dataloaders[phase]:
                #inputs = inputs.to(device)
                #labels = labels.to(device)
                #inputs = inputs.unsqueeze(0)
                ctr += 1
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                if ctr % 100 == 0:
                    print('[' + str(ctr) + '/' + str(dataset_sizes[phase]) + '] => loss: ' + str(running_loss/ctr))

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 12)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=2)